In [1]:
import scanpy as sc
import os
import anndata as ad
from tqdm import tqdm
from mygene import MyGeneInfo
from lib.data_manager import tokenize_h5ad

In [2]:
def convert_gene_names(genes_symbol):
    mg = MyGeneInfo()
    # Prende i primi 10 geni unici
    query_result = mg.querymany(genes_symbol, scopes="symbol", fields="ensembl.gene", species="human")
    conversions = {}
    for q in query_result:
        if  q['query'] in conversions.keys() or 'ensembl' not in q.keys():
            continue
        if isinstance(q['ensembl'], list):
            conversions[q['query']] = q['ensembl'][0]['gene']
        else:
            conversions[q['query']] = q['ensembl']['gene']

    return [  conversions[g] if g in conversions.keys() else 'Not Found'  for g in genes_symbol]


In [5]:
res = []
for fname in tqdm(os.listdir('mtx')):
    if not fname.endswith('.txt'):
        continue
    data = sc.read_csv('mtx/' + fname,delimiter='\t')
    data.var['cell_line'] = fname.split('.')[0]
    res.append(data)

res = ad.concat(res,axis=1)

100%|██████████| 14/14 [00:38<00:00,  2.73s/it]


AnnData object with n_obs × n_vars = 15994 × 8848
    var: 'cell_line'

In [7]:
ensemblids = convert_gene_names(list(res.obs.index))
res1 = res[[c != 'Not Found' for c in ensemblids],:]
res1.obs['ensembl_id'] = [ c for c in ensemblids if c != 'Not Found' ]
res1.var['n_counts'] = res1.X.sum(axis=0)

Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
200 input query terms found dup hits:	[('LINC00115', 2), ('SLC35E2A', 2), ('TNFRSF14-AS1', 3), ('CAMTA1-DT', 2), ('LINC01355', 3), ('NFYC-
2356 input query terms found no hit:	['AL390719.2', 'AL645728.1', 'AL691432.2', 'FO704657.1', 'AL109917.1', 'AL590822.2', 'AL139246.5', '
/tmp/ipykernel_15063/3829315464.py:3: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  res1.obs['ensembl_id'] = [ c for c in ensemblids if c != 'Not Found' ]


In [30]:
res1.T.write_h5ad('merged/rna-seq.h5ad')

In [31]:
tokenize_h5ad('merged', 'tokenized')

Tokenizing merged/rna-seq.h5ad


/home/jerko/Projects/DREAMS2/venv/lib/python3.13/site-packages/geneformer/tokenizer.py:495: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  for i in adata.var["ensembl_id_collapsed"][coding_miRNA_loc]
/home/jerko/Projects/DREAMS2/venv/lib/python3.13/site-packages/geneformer/tokenizer.py:498: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coding_miRNA_ids = adata.var["ensembl_id_collapsed"][coding_miRNA_loc]


merged/rna-seq.h5ad has no column attribute 'filter_pass'; tokenizing all cells.
Creating dataset.
